In [2]:
import pandas as pd
import numpy as np
import talib

from plotly.subplots import make_subplots

import plotly.graph_objects as go

In [3]:
import yfinance as yf
import os

In [ ]:
msft = yf.Ticker("MSFT")

# get stock info
msft.info

# get historical market data
hist = msft.history(period="max")

In [ ]:
data = yf.download("AAPL")

In [ ]:
data.reset_index()

In [ ]:
def assignFactor(ticker, training=True):
#     df = pd.read_csv(filename)
    try:
        df = yf.download(ticker)

        df.reset_index(inplace=True)

        df.Date = pd.to_datetime(df.Date)

        df.sort_values(by='Date', inplace=True)

        df['sma_15'] = talib.SMA(df['Adj Close'], timeperiod=15)

        df['sma_15_diff'] = df['sma_15'].diff()

        df['Trend'] = 'no'

        start_ = min(df.index) + 20

        for d, row in df[df.index > start_].iterrows():
            if sum(df[(df.index <= d) & (df.index > d - 5)].sma_15_diff < 0) == 5 and row['sma_15'] > row['Adj Close']:
                df.loc[d, 'Trend'] = 'down'
            elif sum(df[(df.index <= d) & (df.index > d - 5)].sma_15_diff > 0) == 5 and row['sma_15'] < row['Adj Close']:
                df.loc[d, 'Trend'] = 'up'

        df['Trading Signal'] = 0

        hold = 0

        for d, row in df[df.index > start_].iterrows():
            try:
                min_ = min(df[(df.index >= d) & (df.index < d + 3)]['Adj Close'])
                max_ = max(df[(df.index >= d) & (df.index < d + 3)]['Adj Close'])
            except:
                print("Can't lood ahead")
                break
            if max_ != min_:
                if row['Trend'] == 'up':
                    hold = 0.5
                    df.loc[d, 'Trading Signal'] = (row['Adj Close'] - min_)*0.5/(max_ - min_) + hold

                elif row['Trend'] == 'down':
                    hold = 0
                    df.loc[d, 'Trading Signal'] = (row['Adj Close'] - min_)*0.5/(max_ - min_) + hold
                else:
                    df.loc[d, 'Trading Signal'] = (row['Adj Close'] - min_)*0.5/(max_ - min_) + hold

        macd, macdsignal, macdhist = talib.MACD(df['Adj Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        df['macd'] = macd
        df['macdsignal'] = macdsignal

        slowk, slowd = talib.STOCH(df.High.values, df.Low.values, df.Close.values, 5, 3, 0, 3, 0)

        df['slowk'], df['slowd'] = slowk, slowd

        df['RSI'] = talib.RSI(df['Adj Close'])

        df['WILLR'] = talib.WILLR(df.High.values, df.Low.values, df.Close.values)

        df.dropna(inplace=True)
        
        df.to_csv('stockData/' + ticker + '.csv', index=False)

        return df
    
    except:
#         print("Can't be done for {}".format(ticker))
        return None

In [ ]:
df_ticker_list = pd.read_csv('/home/usman/Downloads/ticker_list.csv')

tickers = df_ticker_list.Ticker.values

In [ ]:
tickers

In [ ]:
# df = assignFactor('AAPL.csv')

# df = df.append(assignFactor('/home/usman/Downloads/BB.csv'))
# df = df.append(assignFactor('/home/usman/Downloads/FE.csv'))
# df = df.append(assignFactor('/home/usman/Downloads/AMD.csv'))
# df = df.append(assignFactor('/home/usman/Downloads/INTC.csv'))
# df = df.append(assignFactor('/home/usman/Downloads/NVDA.csv'))
# df = df.append(assignFactor('/home/usman/Downloads/DK.csv'))


# df = assignFactor(tickers[0])
# df['symbol'] = tickers[0]

df = pd.read_csv('stockData/A.csv')

for i in range(1, len(tickers)):
    print(tickers[i])
    if os.path.exists('stockData/' + tickers[i] + '.csv') == False:
        df_hold = assignFactor(tickers[i])
        if type(df_hold) == pd.core.frame.DataFrame and len(df_hold) > 0:
            df_hold['symbol'] = tickers[0]
            df = df.append(df_hold)

In [4]:
import glob

In [5]:
files = glob.glob('stockData/*csv')[:1000]

In [6]:
df = pd.read_csv(files[0])

for i, f in enumerate(files[1:]):
    print(i+2, f)
    df = df.append(pd.read_csv(f))

2 stockData/FEMS.csv
3 stockData/HRTG.csv
4 stockData/ESXB.csv
5 stockData/AOA.csv
6 stockData/GDL.csv
7 stockData/BLV.csv
8 stockData/AMED.csv
9 stockData/HTLFP.csv
10 stockData/HII.csv
11 stockData/ASPU.csv
12 stockData/CCXI.csv
13 stockData/INVA.csv
14 stockData/DUK.csv
15 stockData/FRT.csv
16 stockData/BLDP.csv
17 stockData/EUO.csv
18 stockData/CHCI.csv
19 stockData/AUTL.csv
20 stockData/GBT.csv
21 stockData/JPXN.csv
22 stockData/GTS.csv
23 stockData/ABT.csv
24 stockData/CIO.csv
25 stockData/FRSX.csv
26 stockData/FMK.csv
27 stockData/IDMO.csv
28 stockData/CELP.csv
29 stockData/IGIB.csv
30 stockData/FANH.csv
31 stockData/DQ.csv
32 stockData/GJO.csv
33 stockData/ATXI.csv
34 stockData/CNSP.csv
35 stockData/EWU.csv
36 stockData/BSX.csv
37 stockData/HXL.csv
38 stockData/HHR.csv
39 stockData/CMSA.csv
40 stockData/GNE.csv
41 stockData/EZJ.csv
42 stockData/JHCS.csv
43 stockData/FNGO.csv
44 stockData/CFFI.csv
45 stockData/ESGR.csv
46 stockData/JHB.csv
47 stockData/DNN.csv
48 stockData/APOG.

369 stockData/IBTH.csv
370 stockData/BLOK.csv
371 stockData/HUGE.csv
372 stockData/EIG.csv
373 stockData/ETTX.csv
374 stockData/DMXF.csv
375 stockData/AMT.csv
376 stockData/ATHE.csv
377 stockData/AESE.csv
378 stockData/GSG.csv
379 stockData/GSBC.csv
380 stockData/HCCOU.csv
381 stockData/FIV.csv
382 stockData/EBR.csv
383 stockData/FLT.csv
384 stockData/IDA.csv
385 stockData/DBS.csv
386 stockData/IBN.csv
387 stockData/HA.csv
388 stockData/CASH.csv
389 stockData/CTY.csv
390 stockData/CCO.csv
391 stockData/FRME.csv
392 stockData/EDV.csv
393 stockData/AVNW.csv
394 stockData/AGS.csv
395 stockData/CHU.csv
396 stockData/FUND.csv
397 stockData/FEYE.csv
398 stockData/FSEA.csv
399 stockData/IPGP.csv
400 stockData/CPAAU.csv
401 stockData/DMRE.csv
402 stockData/BRID.csv
403 stockData/KELYB.csv
404 stockData/ADRO.csv
405 stockData/AXTA.csv
406 stockData/CGA.csv
407 stockData/CRM.csv
408 stockData/AEMD.csv
409 stockData/CHAD.csv
410 stockData/GES.csv
411 stockData/FFHL.csv
412 stockData/AFH.csv
413 s

732 stockData/EMB.csv
733 stockData/FUSE.csv
734 stockData/AVB.csv
735 stockData/CSML.csv
736 stockData/HIO.csv
737 stockData/FLMB.csv
738 stockData/FMX.csv
739 stockData/IYH.csv
740 stockData/FNLC.csv
741 stockData/ADBE.csv
742 stockData/GP.csv
743 stockData/ARYBU.csv
744 stockData/HDG.csv
745 stockData/FVE.csv
746 stockData/CNHI.csv
747 stockData/EMKR.csv
748 stockData/B.csv
749 stockData/DE.csv
750 stockData/BLBD.csv
751 stockData/H.csv
752 stockData/FRG.csv
753 stockData/CAJ.csv
754 stockData/BLD.csv
755 stockData/CI.csv
756 stockData/FSFG.csv
757 stockData/DAX.csv
758 stockData/ITRI.csv
759 stockData/AGNC.csv
760 stockData/FLCO.csv
761 stockData/JPM.csv
762 stockData/GMDA.csv
763 stockData/FTOCU.csv
764 stockData/ADVM.csv
765 stockData/CMS.csv
766 stockData/BRZU.csv
767 stockData/JTD.csv
768 stockData/ATGE.csv
769 stockData/JAGG.csv
770 stockData/ESGS.csv
771 stockData/FENY.csv
772 stockData/ADT.csv
773 stockData/EYEN.csv
774 stockData/CYTK.csv
775 stockData/HDGE.csv
776 stockData

In [7]:
len(df)

3402265

In [8]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sma_15,sma_15_diff,Trend,Trading Signal,macd,macdsignal,slowk,slowd,RSI,WILLR
0,2010-05-18,27.180000,27.180000,25.870001,26.070000,18.719772,12300.0,20.321044,-0.233608,down,0.50000,-1.126530,-1.046674,15.881325,38.378813,34.398507,-80.025451
1,2010-05-19,25.790001,25.790001,24.760000,25.570000,18.360744,9500.0,20.070681,-0.250363,down,0.50000,-1.185363,-1.074412,22.031663,25.671497,32.670357,-83.538466
2,2010-05-20,24.540001,24.540001,23.680000,23.900000,17.161591,8000.0,19.726971,-0.343710,down,0.00000,-1.313607,-1.122251,14.144961,17.352650,27.670180,-96.636096
3,2010-05-21,23.959999,25.260000,23.500000,25.200001,18.095066,6900.0,19.486661,-0.240310,down,0.50000,-1.324648,-1.162730,23.086276,19.754300,35.895162,-70.740087
4,2010-05-24,25.430000,25.430000,25.020000,25.020000,17.965815,2900.0,19.253053,-0.233608,down,0.23973,-1.328514,-1.195887,30.704057,22.645098,35.296656,-73.097335


In [9]:
df.to_csv('trainData.csv', index=False)

In [10]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

In [11]:
data = df[(df.Date > '2016-01-01') & (df.Date < '2020-01-01')][['sma_15', 'macd', 'macdsignal', 'slowk', 'slowd', 'RSI', 'WILLR']]

scaler.fit(data)

import pickle

with open("sclar.pickle", "wb") as file_:
    pickle.dump(scaler, file_, -1)

X = scaler.transform(data)

y = df[(df.Date > '2016-01-01') & (df.Date < '2020-01-01')]['Trading Signal'].values

In [13]:
len(X)

771856

In [14]:
test = df[df.Date >= '2020-01-01'][['sma_15', 'macd', 'macdsignal', 'slowk', 'slowd', 'RSI', 'WILLR']]

X_test = scaler.transform(test)

y_test = df[df.Date >= '2020-01-01']['Trading Signal'].values

In [15]:
import numpy as np
import time


def _mean_squared_error(y, pred):
    return 0.5 * np.mean((y - pred) ** 2)


def _mean_abs_error(y, pred):
    return np.mean(np.abs(y, pred))


def _sigmoid(x):
    return 1. / (1. + np.exp(-x))

def _tanh(x):
    return np.tanh(x)

def _fourier(x):
    return np.sin(x)


def _hardlimit(x):
    return (x >= 0).astype(int)


def _identity(x):
    return x


def getActivation(name):
    return {
        'sigmoid': _sigmoid,
        'fourier': _fourier,
        'hardlimit': _hardlimit,
        'tanh': _tanh
    }[name]


def getLoss(name):
    return {
        'mse': _mean_squared_error,
        'mae': _mean_abs_error
    }[name]


class ELM:
    def __init__(self, num_input_nodes, num_hidden_units, num_out_units, activation='sigmoid',
                 loss='mse', beta_init=None, w_init=None, bias_init=None):
        self._num_input_nodes = num_input_nodes
        self._num_hidden_units = num_hidden_units
        self._num_out_units = num_out_units

        self._activation = getActivation(activation)
        self._loss = getLoss(loss)

        if isinstance(beta_init, np.ndarray):
            self._beta = beta_init
        else:
            self._beta = np.random.uniform(-1., 1., size=(self._num_hidden_units, self._num_out_units))

        if isinstance(w_init, np.ndarray):
            self._w = w_init
        else:
            self._w = np.random.uniform(-1, 1, size=(self._num_input_nodes, self._num_hidden_units))

        if isinstance(bias_init, np.ndarray):
            self._bias = bias_init
        else:
            self._bias = np.zeros(shape=(self._num_hidden_units,))

        print('Bias shape:', self._bias.shape)
        print('W shape:', self._w.shape)
        print('Beta shape:', self._beta.shape)

    def fit(self, X, Y, display_time=False):
        H = self._activation(X.dot(self._w) + self._bias)

        # Moore–Penrose pseudo inverse
        if display_time:
            start = time.time()
        H_pinv = np.linalg.pinv(H)
        if display_time:
            stop = time.time()
            print(f'Train time: {stop-start}')

        self._beta = H_pinv.dot(Y)

        # print('Fit Beta shape:', self._beta.shape)

    def __call__(self, X):
        H = self._activation(X.dot(self._w) + self._bias)
        return H.dot(self._beta)

    def evaluate(self, X, Y):
        pred = self(X)

        # Loss (base on model setting)
        loss = self._loss(Y, pred)

        # Accuracy
        acc = np.sum(np.argmax(pred, axis=-1) == np.argmax(Y, axis=-1)) / len(Y)

        # Unweighted Average Recall
        # TODO

        return loss, acc

In [17]:
best_hidden_layer = 0
best_train_loss = 1e9
best_activation = None
activations = ['sigmoid', 'fourier', 'hardlimit', 'tanh']

input_length = 7
num_classes = 1


for act in activations:
    print('activation: ', act)
    for i in range(3, 30):
        print('hiddent_layer: ', i)
        num_hidden_layers = i
        model = ELM(
            input_length,
            num_hidden_layers,
            num_classes, activation=act, bias_init=np.random.rand((num_hidden_layers))
        )
        
        # Train
        model.fit(X, y, display_time=True)
        train_loss, train_acc = model.evaluate(X_test, y_test)

        if train_loss < best_train_loss:
            best_train_loss = train_loss
            best_hidden_layer = i
            best_activation = act

activation:  sigmoid
hiddent_layer:  3
Bias shape: (3,)
W shape: (7, 3)
Beta shape: (3, 1)
Train time: 0.08255410194396973
hiddent_layer:  4
Bias shape: (4,)
W shape: (7, 4)
Beta shape: (4, 1)
Train time: 0.11990237236022949
hiddent_layer:  5
Bias shape: (5,)
W shape: (7, 5)
Beta shape: (5, 1)
Train time: 0.15575170516967773
hiddent_layer:  6
Bias shape: (6,)
W shape: (7, 6)
Beta shape: (6, 1)
Train time: 0.20953011512756348
hiddent_layer:  7
Bias shape: (7,)
W shape: (7, 7)
Beta shape: (7, 1)
Train time: 0.29536938667297363
hiddent_layer:  8
Bias shape: (8,)
W shape: (7, 8)
Beta shape: (8, 1)
Train time: 0.31659865379333496
hiddent_layer:  9
Bias shape: (9,)
W shape: (7, 9)
Beta shape: (9, 1)
Train time: 0.34644198417663574
hiddent_layer:  10
Bias shape: (10,)
W shape: (7, 10)
Beta shape: (10, 1)
Train time: 0.43500351905822754
hiddent_layer:  11
Bias shape: (11,)
W shape: (7, 11)
Beta shape: (11, 1)
Train time: 0.5083446502685547
hiddent_layer:  12
Bias shape: (12,)
W shape: (7, 12)


Train time: 2.2374045848846436
hiddent_layer:  28
Bias shape: (28,)
W shape: (7, 28)
Beta shape: (28, 1)
Train time: 2.3532562255859375
hiddent_layer:  29
Bias shape: (29,)
W shape: (7, 29)
Beta shape: (29, 1)
Train time: 2.3948678970336914
activation:  tanh
hiddent_layer:  3
Bias shape: (3,)
W shape: (7, 3)
Beta shape: (3, 1)
Train time: 0.09632658958435059
hiddent_layer:  4
Bias shape: (4,)
W shape: (7, 4)
Beta shape: (4, 1)
Train time: 0.1259756088256836
hiddent_layer:  5
Bias shape: (5,)
W shape: (7, 5)
Beta shape: (5, 1)
Train time: 0.1704263687133789
hiddent_layer:  6
Bias shape: (6,)
W shape: (7, 6)
Beta shape: (6, 1)
Train time: 0.2326030731201172
hiddent_layer:  7
Bias shape: (7,)
W shape: (7, 7)
Beta shape: (7, 1)
Train time: 0.2818410396575928
hiddent_layer:  8
Bias shape: (8,)
W shape: (7, 8)
Beta shape: (8, 1)
Train time: 0.3733491897583008
hiddent_layer:  9
Bias shape: (9,)
W shape: (7, 9)
Beta shape: (9, 1)
Train time: 0.42964887619018555
hiddent_layer:  10
Bias shape: (

In [18]:
print(best_train_loss)
print(best_hidden_layer)
print(best_activation)

0.04428135286857489
28
tanh


In [19]:
input_length = 7
num_hidden_layers = best_hidden_layer
num_classes = 1

model = ELM(
    input_length,
    num_hidden_layers,
    num_classes, activation=best_activation, bias_init=np.random.rand((num_hidden_layers))
)

# Train
model.fit(X, y, display_time=True)
train_loss, train_acc = model.evaluate(X, y)
print('train loss: %f' % train_loss)
print('train acc: %f' % train_acc)

Bias shape: (28,)
W shape: (7, 28)
Beta shape: (28, 1)
Train time: 2.294771432876587
train loss: 0.044693
train acc: 0.000000


In [20]:
# import pickle

with open("ELM.pickle", "wb") as file_:
    pickle.dump(model, file_, -1)

In [21]:
# import pickle 

# model = pickle.load(open("ELM.pickle", "rb", -1))

In [86]:
good_stocks = []
for f in glob.glob('stockData/*csv'):
    if f not in files:
        df_test = pd.read_csv(f)
        df_test = df_test[df_test.Date >= '2020-01-01']
        if len(df_test) > 0:
#             print(f)
            df_test.Date = pd.to_datetime(df_test.Date)
            
            test = df_test[['sma_15', 'macd', 'macdsignal', 'slowk', 'slowd', 'RSI', 'WILLR']]

            X_test = scaler.transform(test)

            y_test = df_test['Trading Signal'].values

            pred = model.__call__(X_test)
            
            if pred[-1] < 0.1:
                print('try: ', f)

            mse = 0.5 * np.mean((y_test - pred) ** 2)
#             print(f, mse)
            if mse < 0.04:
                good_stocks.append(f)

try:  stockData/FSRVU.csv
try:  stockData/CFFAU.csv
try:  stockData/DNRCQ.csv
try:  stockData/ASLN.csv
try:  stockData/ITRM.csv
try:  stockData/DWCR.csv
try:  stockData/EVGBC.csv
try:  stockData/ERH.csv
try:  stockData/IMAC.csv
try:  stockData/EVLMC.csv
try:  stockData/CIEN.csv
try:  stockData/DGX.csv
try:  stockData/CCOI.csv


In [33]:
print(len(good_stocks))
good_stocks

283


['stockData/BNDX.csv',
 'stockData/EMQQ.csv',
 'stockData/KIM.csv',
 'stockData/FLZA.csv',
 'stockData/AMRC.csv',
 'stockData/IDEV.csv',
 'stockData/IWF.csv',
 'stockData/EMXC.csv',
 'stockData/HWCPZ.csv',
 'stockData/GNTX.csv',
 'stockData/EAI.csv',
 'stockData/HAWX.csv',
 'stockData/KL.csv',
 'stockData/JE.csv',
 'stockData/CE.csv',
 'stockData/BHAT.csv',
 'stockData/CAPL.csv',
 'stockData/AMTX.csv',
 'stockData/CELC.csv',
 'stockData/DXJS.csv',
 'stockData/HTRB.csv',
 'stockData/ARES.csv',
 'stockData/APO.csv',
 'stockData/CQQQ.csv',
 'stockData/IWV.csv',
 'stockData/FLKR.csv',
 'stockData/ASG.csv',
 'stockData/ALX.csv',
 'stockData/IXN.csv',
 'stockData/AGI.csv',
 'stockData/FFA.csv',
 'stockData/AADR.csv',
 'stockData/HPQ.csv',
 'stockData/BKEM.csv',
 'stockData/GDDY.csv',
 'stockData/BMRN.csv',
 'stockData/HGLB.csv',
 'stockData/AIO.csv',
 'stockData/IQSI.csv',
 'stockData/DKNG.csv',
 'stockData/BBSI.csv',
 'stockData/CLIX.csv',
 'stockData/JEPI.csv',
 'stockData/GGT.csv',
 'stoc

In [87]:
df_test = pd.read_csv('stockData/CIEN.csv')

df_test.Date = pd.to_datetime(df_test.Date)

df_test = df_test[df_test.Date >= '2020-01-01']

In [88]:
df_test.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sma_15,sma_15_diff,Trend,Trading Signal,macd,macdsignal,slowk,slowd,RSI,WILLR
5729,2020-01-02,43.250000,43.389999,42.450001,42.860001,42.860001,1470900,41.653333,0.544667,up,1.000000,1.452638,1.250546,60.668491,70.983102,69.419109,-18.500010
5730,2020-01-03,42.330002,42.700001,41.840000,41.869999,41.869999,1354700,42.084000,0.430667,no,0.970588,1.350756,1.270588,36.441858,55.959798,60.428474,-59.490106
5731,2020-01-06,41.570000,42.160000,41.500000,41.889999,41.889999,1202400,42.035333,-0.048667,no,0.732877,1.257137,1.267898,24.345897,40.485415,60.539663,-50.847444
5732,2020-01-07,41.730000,41.950001,41.540001,41.549999,41.549999,1163000,42.046666,0.011333,no,0.500000,1.142339,1.242786,8.405255,23.064337,57.577852,-62.372870
5733,2020-01-08,42.650002,43.290001,42.250000,42.279999,42.279999,2471600,42.115333,0.068667,no,1.000000,1.097614,1.213751,21.516716,18.089289,61.889026,-40.363659


In [89]:
test = df_test[['sma_15', 'macd', 'macdsignal', 'slowk', 'slowd', 'RSI', 'WILLR']]

X_test = scaler.transform(test)

y_test = df_test['Trading Signal'].values

In [90]:
pred = model.__call__(X_test)

0.5 * np.mean((y_test - pred) ** 2)

0.03923369366745286

In [91]:
df_test['pred'] = pred

In [92]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=df_test['Date'], y=df_test['Trading Signal'],
                    mode='lines',
                    name='Trading Signal'))

# Add traces
fig.add_trace(go.Scatter(x=df_test['Date'], y=df_test['pred'],
                    mode='lines',
                    name='Predictions'))


fig.show()

In [93]:
def buysellSignal(x):
    if x > 0.7:
        return 'SELL'
    if x < 0.35:
        return 'BUY'
    
df_test['action'] = df_test['pred'].apply(lambda x: buysellSignal(x))

In [94]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_test['Date'], y=df_test['Adj Close'],
                    mode='lines',
                    name='markers', hovertext=df_test['pred']))

fig.add_trace(go.Scatter(x=df_test[df_test['action'] == 'SELL'].Date, y=df_test[df_test['action'] == 'SELL']['Adj Close'],
                    mode='markers',
                    name='SELL', hovertext=df_test[df_test['action'] == 'SELL']['pred']))

fig.add_trace(go.Scatter(x=df_test[df_test['action'] == 'BUY'].Date, y=df_test[df_test['action'] == 'BUY']['Adj Close'],
                    mode='markers',
                    name='BUY', hovertext=df_test[df_test['action'] == 'BUY']['pred']))


fig.show()